In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from IPython.display import HTML
import sys
from PIL import Image

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    # move tensors to a device
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

device = get_default_device()
device

In [ ]:
mkdir healthy multiple_diseases rust scab main


In [ ]:
mkdir healthy/images scab/images multiple_diseases/images rust/images main/images

In [ ]:
# linear FOV = 2 tan(AOV/2) * Height of the Drone
# AOV for DJI Mavic 2 Pro is 77 degrees, we are going to assume this is horizontal view
height = 20 # 200 feet is half of the max altitude allowed by the FAA 
horizontal_FOV = 2 * .795 * height
vertical_FOV = horizontal_FOV * 3648/5472
horizontal_FOV, vertical_FOV # this leads to a picture that covers 1.5 acres. 

In [ ]:
ppi = 5472/horizontal_FOV
ppi = ppi/12

ppi

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
#test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
#sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')
#print(train.head())
#train_label = train.loc[:, 'healthy':].values

healthy = train['healthy'].values
multiple_diseases = train['multiple_diseases'].values
rust = train['rust'].values
scab = train['scab'].values


In [ ]:

import shutil
# first arg for shutil.copy() is the file to copy, second is where u want to copy into (you'll prob need to change the file you want to copy)
healthyy = []
scabb = []
rustt = []
multiple = []
imageNum = [healthyy, scabb, rustt, multiple]
for i in range(train['image_id'].values.size):
    if healthy[i] == 1:
        shutil.copy(f'/kaggle/input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg', f'healthy/images/Train_{i}.jpg')
        imageNum[0].append(i)
    if scab[i] == 1:
        shutil.copy(f'/kaggle/input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg', f'scab/images/Train_{i}.jpg')
        imageNum[1].append(i)
    if rust[i] == 1:
        shutil.copy(f'/kaggle/input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg', f'rust/images/Train_{i}.jpg')
        imageNum[2].append(i)
    if multiple_diseases[i] == 1:
        shutil.copy(f'/kaggle/input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg', f'multiple_diseases/images/Train_{i}.jpg')
        imageNum[3].append(i)
    shutil.copy(f'/kaggle/input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg', f'main/images/Train_{i}.jpg')
    
for i in range (4):
    print(len(imageNum[i]))

In [ ]:
def fill():
    healthyy = []
    scabb = []
    rustt = []
    multiple = []
    imageNummm = [healthyy, scabb, rustt, multiple]
    for i in range(train['image_id'].values.size):
        if healthy[i] == 1:
            imageNummm[0].append(i)
        if scab[i] == 1:
            imageNummm[1].append(i)
        if rust[i] == 1:
            imageNummm[2].append(i)
        if multiple_diseases[i] == 1:
            imageNummm[3].append(i)
#     for i in range(4):
#         print(len(imageNummm[i]))
    return imageNummm

In [ ]:
print(imageNum[3][5]) #should be 74

In [ ]:
def dehance(image):
    baseheight = (int)(3 * ppi)
    test_image1 = Image.open(image)
    hpercent = (baseheight / float(test_image1.size[1]))
    wsize = int((float(test_image1.size[0]) * float(hpercent)))
    test_image1 = test_image1.resize((wsize, baseheight), Image.ANTIALIAS)
    test_image1.save(image)
    imgplot = plt.imshow(test_image1)
#     print(image)
    return test_image1

In [ ]:
imageNumm = fill()

In [ ]:
def pickNext(imageNumm):
    num = random.randint(0, 4)
    if(num == 4):
        soil = Image.open("../input/soiloverhead/soiloverhead.jpg")
        x, y = random.randint(0, 1856), random.randint(0, 1038)
        box = (x, y, x+64, y+43)
        soilBox = soil.crop(box)
        return num, soilBox
    while(len(imageNumm[num]) == 1):
        print(f"{num} is out of values")
        num = random.randint(0, 2)
        imageNumm = fill()
#     print(f"{num} has enough values")
    image = random.choice(imageNumm[num])
    imageNumm[num].remove(image)
    return num, (f'main/images/Train_{image}.jpg')


    
    
    

In [ ]:
# importing the libraries

import pandas as pd
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage

In [ ]:
n = 4
augments = []
while(n == 4):
    n, image = pickNext(imageNumm)
imagee = imread(image)
augments.append(imagee)
augments.append(rotate(imagee, angle=45, mode = 'wrap'))
augments.append(np.fliplr(imagee))
augments.append(np.flipud(imagee))
augments.append(random_noise(imagee,var=0.2**2))
fig,ax = plt.subplots(nrows=1,ncols=5,figsize=(250,250))
for i in range(5):
    ax[i].imshow(augments[i])
    ax[i].axis('off')

In [ ]:

import random

imageNumm = fill()
def build():
    
    imageNumm = fill()
    print(f"started build")
#     for i in range(4):
#         print(len(imageNumm[i]))
    labels = {'id': [], 'bbox': [], 'sick': []} #0 is healthy, 1 is sick, 2 is background
    canvas_size = (1792, 1204) # this is the resolution of the DJI Mavic 2 Pro camera
    canvas = Image.new('RGB', canvas_size, None)
    imgplot = plt.imshow(canvas)
    current_x = 0
    current_y = 0
    print("initalized build")

    while((current_x, current_y) < (28, 28)):
#         num = random.randint(0, 3)
        num, image = pickNext(imageNumm)
        if(num != 4):
            current_image = dehance(image)
            labels['sick'].append(0 if num == 0 else 1)
            labels['id'].append(image[12:])
            labels['bbox'].append([current_x*64, current_y*43, current_image.size[0], current_image.size[1]])
        else:
            current_image = image
        box = (current_x*64, current_y*43, current_x*64 + current_image.size[0], current_y*43 + current_image.size[1])
        canvas.paste(current_image, box)
        current_x += 1 if current_x < 28 else -28
        if current_x == 0: current_y += 1
    
#     imgplot = plt.imshow(canvas)
    print("finished build")
    return pd.DataFrame(data = labels), canvas
    


In [ ]:
mkdir output

In [ ]:
xx = 0 #1732

In [ ]:
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)
for i in range(xx, xx+600):
    df, canvas = build()
    df.to_csv(f'output/test{i}.csv', index=False,)  
    canvas.save(f'output/test{i}.jpg')
    print(f"done with {i}")
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    


In [ ]:
# import time
# t = time.localtime()
# current_time = time.strftime("%H:%M:%S", t)
# print(current_time)
# for i in range(2):
#     df, canvas = build()
#     df.to_csv(f'output/out{i+2}.csv', index=False,)  
#     canvas.save(f'output/out{i}.jpg')
#     print(f"done with {i+2}")
#     t = time.localtime()
#     current_time = time.strftime("%H:%M:%S", t)
#     print(current_time)

In [ ]:

import os, shutil
def make_archive(source, destination):
        base = os.path.basename(destination)
        name = base.split('.')[0]
        format = base.split('.')[1]
        archive_from = os.path.dirname(source)
        archive_to = os.path.basename(source.strip(os.sep))
        print(source, destination, archive_from, archive_to)
        shutil.make_archive(name, format, archive_from, archive_to)
        shutil.move('%s.%s'%(name,format), destination)

make_archive('./output', 'test.zip')

print("done with zip")

# dir_path = '/healthy'
# try:
#     shutil.rmtree(dir_path)
# except OSError as e:
#     print("Error: %s : %s" % (dir_path, e.strerror))
# dir_path = '/rust'
# try:
#     shutil.rmtree(dir_path)
# except OSError as e:
#     print("Error: %s : %s" % (dir_path, e.strerror))
# dir_path = '/multiple_diseases'    
# try:
#     shutil.rmtree(dir_path)
# except OSError as e:
#     print("Error: %s : %s" % (dir_path, e.strerror))
# dir_path = '/scab'
# try:
#     shutil.rmtree(dir_path)
# except OSError as e:
#     print("Error: %s : %s" % (dir_path, e.strerror))

In [ ]:
import time
#started at 9:53 pm
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)